In [ ]:
import asyncio
from hume import HumeStreamClient
from hume.models.config import LanguageConfig

samples = [
    "Mary had a little lamb,",
    "Its fleece was white as snow.",
    "Everywhere the child went,",
    "The little lamb was sure to go."
]

async def main():
    client = HumeStreamClient(hume_api)
    config = LanguageConfig()
    async with client.connect([config]) as socket:
        for sample in samples:
            result = await socket.send_text(sample)
            emotions = result["language"]["predictions"][0]["emotions"]
            print(emotions)

# 检查是否已存在事件循环
loop = asyncio.get_event_loop()
if loop.is_running():
    # 如果事件循环已经在运行，则使用已有的循环
    loop.create_task(main())
else:
    # 如果没有事件循环在运行，则正常使用 asyncio.run
    asyncio.run(main())


In [2]:
import asyncio
import json
from hume import HumeStreamClient
from hume.models.config import LanguageConfig

hume_api = "bGKD8LrwTrKIHS6BQFVV8p8rrImljoCTiAQik7Yp2Phk3GiV" #需要换成你的API密钥
top_n = 10  # 保留的情感数量
all_emotions= []  # 创建一个空列表来存储所有的情感数据


file_path = input("Enter the path to the text file: ") #读取断句文件
with open(file_path, 'r', encoding='utf-8') as file:
    sentences = [line.strip() for line in file] #删除空行

print("Total number of sentences:", len(sentences)) #打印所有句子数量

try:
    start = int(input("Enter the start index (1-based): ")) #输入需要读取的首句
    end = int(input("Enter the end index (1-based): ")) #输入需要读取的末句
    
    if not 1 <= start <= len(sentences) or not 1 <= end <= len(sentences) or start > end:
        raise ValueError("Invalid range!") #若输入数字不符合要求则报错

    samples = sentences[start - 1:end] #替换samples
except ValueError as ve: #反之，都报错
    print("Invalid input:", ve)
except IndexError:
    
    print("Index out of range!")
except Exception as e:
    print("An error occurred:", str(e))

async def main(top_n,all_emotions):
    client = HumeStreamClient(hume_api)
    config = LanguageConfig()
    async with client.connect([config]) as socket:
        for sample in samples:
            result = await socket.send_text(sample)
            emotions = result["language"]["predictions"][0]["emotions"]
            # 使用top_n变量来控制保留的情感数量
            top_emotions = sorted(emotions, key=lambda x: x['score'], reverse=True)[:top_n]
            all_emotions.append({sample: top_emotions})  # 将每个样本的top N情感数据添加到列表中
            print(top_emotions)
    
    # 将所有情感数据写入JSON文件
    with open('top_emotions.json', 'w') as f:
        json.dump(all_emotions, f, ensure_ascii=False, indent=4)  # 确保使用非ASCII字符和格式化输出

# 检查是否已存在事件循环
loop = asyncio.get_event_loop()
if loop.is_running():
    # 如果事件循环已经在运行，则使用已有的循环
    loop.create_task(main(top_n,all_emotions))  # 传入你想要的截取数量，例如10
else:
    # 如果没有事件循环在运行，则正常使用 asyncio.run
    asyncio.run(main(top_n,all_emotions))  # 同样传入截取数量

c:\Users\qnj\anaconda3\envs\emotion\Lib\site-packages\pydantic\_internal\_fields.py:160: UserWarning: Field "model_provider" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\qnj\anaconda3\envs\emotion\Lib\site-packages\pydantic\_internal\_fields.py:160: UserWarning: Field "model_resource" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Total number of sentences: 1595


[{'name': 'Boredom', 'score': 0.16840359568595886}, {'name': 'Annoyance', 'score': 0.09857567399740219}, {'name': 'Calmness', 'score': 0.09760017693042755}, {'name': 'Confusion', 'score': 0.08876815438270569}, {'name': 'Concentration', 'score': 0.07571996748447418}, {'name': 'Surprise (negative)', 'score': 0.05172520503401756}, {'name': 'Awkwardness', 'score': 0.051327385008335114}, {'name': 'Realization', 'score': 0.044413913041353226}, {'name': 'Satisfaction', 'score': 0.040825918316841125}, {'name': 'Aesthetic Appreciation', 'score': 0.039232369512319565}]
[{'name': 'Confusion', 'score': 0.5000172257423401}, {'name': 'Calmness', 'score': 0.09883999079465866}, {'name': 'Awkwardness', 'score': 0.06649528443813324}, {'name': 'Contemplation', 'score': 0.056936122477054596}, {'name': 'Doubt', 'score': 0.05404890701174736}, {'name': 'Boredom', 'score': 0.04989081248641014}, {'name': 'Interest', 'score': 0.044054627418518066}, {'name': 'Surprise (negative)', 'score': 0.04163090139627457}, 

In [3]:
# 打印所有情感数据,这些数据被组织成了一个列表由字典组成，每个字典包含了一个样本的top N情感数据
all_emotions

[{'1. Chapter 1': [{'name': 'Boredom', 'score': 0.16840359568595886},
   {'name': 'Annoyance', 'score': 0.09857567399740219},
   {'name': 'Calmness', 'score': 0.09760017693042755},
   {'name': 'Confusion', 'score': 0.08876815438270569},
   {'name': 'Concentration', 'score': 0.07571996748447418},
   {'name': 'Surprise (negative)', 'score': 0.05172520503401756},
   {'name': 'Awkwardness', 'score': 0.051327385008335114},
   {'name': 'Realization', 'score': 0.044413913041353226},
   {'name': 'Satisfaction', 'score': 0.040825918316841125},
   {'name': 'Aesthetic Appreciation', 'score': 0.039232369512319565}]},
 {'we are introduced to the narrator, a pilot, and his ideas about grown-ups': [{'name': 'Confusion',
    'score': 0.5000172257423401},
   {'name': 'Calmness', 'score': 0.09883999079465866},
   {'name': 'Awkwardness', 'score': 0.06649528443813324},
   {'name': 'Contemplation', 'score': 0.056936122477054596},
   {'name': 'Doubt', 'score': 0.05404890701174736},
   {'name': 'Boredom', 's